<a href="https://colab.research.google.com/github/Ayushee-Seeburrun/ASAG-with-Data-Augmentation/blob/main/cleaning_balanced_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re, unicodedata

INFILE  = "/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_final.csv"
#OUTFILE = "/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_clean.csv"

def norm_text(s):
    s = str(s)
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00A0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

df = pd.read_csv(INFILE)
df["type"] = df["type"].fillna("").astype(str).str.strip().str.lower()
df["EssayText"] = df["EssayText"].map(norm_text)

orig = df[df["type"] != "paraphrased"].copy()
para = df[df["type"] == "paraphrased"].copy()

#here ^p are removed from the records(paraphrases only)
para = para[~para["EssayText"].str.lower().str.contains(r"\^p", na=False)]

#removing non-english sentences generated by claude during the balancing process
danish_pat = r"([æøå]|\bhvis\b|\bvil\b|\bjeg\b|\bfordi\b|\bder\b|\bdet\b|\bvære\b|\bnår\b|\bikke\b)"
para = para[~para["EssayText"].str.lower().str.contains(danish_pat, regex=True, na=False)]

#remove paraphrase duplicates (per score, normalized)
para["_norm"] = para["EssayText"].map(norm_text)
para = para.drop_duplicates(subset=["Score2", "_norm"], keep="first").drop(columns=["_norm"])

cleaned = pd.concat([orig, para], ignore_index=True)

print("Before:", len(df), "| After:", len(cleaned))
print("Type counts:\n", cleaned["type"].value_counts(dropna=False))
print("Score counts:\n", cleaned["Score2"].value_counts().sort_index())

#cleaned.to_csv(OUTFILE, index=False)
#print("Saved cleaned base dataset to:", OUTFILE)


/tmp/ipython-input-491861055.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  para = para[~para["EssayText"].str.lower().str.contains(danish_pat, regex=True, na=False)]


Before: 17252 | After: 17225
Type counts:
 type
               13765
paraphrased     3460
Name: count, dtype: int64
Score counts:
 Score2
0    5424
1    4518
2    4194
3    3089
Name: count, dtype: int64


In [ ]:
#here - reordering how the records are saved in the dataste, the originals first and then wwe have the paraphrased
import pandas as pd

INFILE  = "/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_clean.csv"
#OUTFILE = "/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_clean.csv"

df = pd.read_csv(INFILE)

df["type"] = df["type"].fillna("").astype(str).str.strip().str.lower()

#splitting  into original and paraphrased
orig = df[df["type"] != "paraphrased"].copy()
para = df[df["type"] == "paraphrased"].copy()

#ensure paraphrased rows have type "paraphrased"
para["type"] = "paraphrased"

#sort paraphrases by score
para = para.sort_values(by=["Score2"], ascending=True).reset_index(drop=True)

#originals first, paraphrases appended afterwards
final_df = pd.concat([orig.reset_index(drop=True), para], ignore_index=True)

#final_df.to_csv(OUTFILE, index=False)

#print("Saved:", OUTFILE)
print("Rows:", len(final_df))
print("Type counts:\n", final_df["type"].value_counts(dropna=False))
print("Score counts:\n", final_df["Score2"].value_counts().sort_index())


Rows: 17225
Type counts:
 type
               13765
paraphrased     3460
Name: count, dtype: int64
Score counts:
 Score2
0    5424
1    4518
2    4194
3    3089
Name: count, dtype: int64


In [ ]:
print("Final cleaned and balanced dataset: \n", cleaned["Score2"].value_counts().sort_index(), "\n", final_df["Score2"].value_counts().sort_index())

Final cleaned and balanced dataset: 
 Score2
0    5424
1    4518
2    4194
3    3089
Name: count, dtype: int64 
 Score2
0    5424
1    4518
2    4194
3    3089
Name: count, dtype: int64
